# Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# Define the needed variables

In [78]:
details ={
    'Aircraft':[],
    'Type Of Traveller':[],
    'Seat Type':[],
    'Route':[],
    'Date Flown':[],
    'Seat Comfort':[],
    'Cabin Staff Service':[],
    'Food & Beverages':[],
    'Inflight Entertainment':[],
    'Ground Service':[],
    'Wifi & Connectivity':[],
    'Value For Money':[],
    'Recommended':[]
}
num=[]
rating=[]
features =["Missing" ,"Missing" , "Missing", "Missing","Missing" , "Missing"]

# Define the first page url to get the number of pages

In [2]:
url ='https://www.airlinequality.com/airline-reviews/british-airways/page/1/'

In [5]:
page = requests.get(url)

In [6]:
src= page.content

In [7]:
soup= BeautifulSoup(src , "lxml")

In [10]:
num_pages =soup.find_all("article" , class_="comp comp_reviews-pagination querylist-pagination position-")

In [12]:
pages =num_pages[0].find_all('a')[-2].text.strip()

In [13]:
pages=int(pages)

In [14]:
pages

385

# Get all links of review pages on the website

In [15]:
links =[]

In [16]:
for i in range(1 , pages+1):
  links.append(f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/")

# Function to split list on (1) to get the star rate reviews

In [49]:
def split_list_on_one(lst):
    sublists = []
    current_sublist = []

    for item in lst:
        if item == 1 and current_sublist:  # Check if item is 1 and current_sublist is not empty
            sublists.append(current_sublist)
            current_sublist = []
        current_sublist.append(item)

    if current_sublist:  # Add the last sublist if not empty
        sublists.append(current_sublist)

    return sublists

# Fill our dictionary with deviews

In [80]:
for link in links:

  page = requests.get(link)
  src= page.content
  soup= BeautifulSoup(src , "lxml")
  review = soup.find_all("div" , class_ ="body")
  for i in range(len(review)):

    # Get the catigorical reviews

    cate_reviews =review[i].find_all('td' , class_="review-value")


    if review[i].find_all('td' , class_="review-rating-header aircraft"):
      features[0] = cate_reviews[0].text.strip()
    else:
      cate_reviews.insert(0 , "Missing")

    if review[i].find_all('td' , class_="review-rating-header type_of_traveller"):
      features[1] = cate_reviews[1].text.strip()
    else:
        cate_reviews.insert(1 , "Missing")


    if review[i].find_all('td' , class_="review-rating-header cabin_flown"):
      features[2] = cate_reviews[2].text.strip()
    else:
        cate_reviews.insert(2 , "Missing")

    if review[i].find_all('td' , class_="review-rating-header route"):
      features[3] = cate_reviews[-3].text.strip()
    else:
        cate_reviews.insert(3 , "Missing")


    if review[i].find_all('td' , class_="review-rating-header date_flown"):
      features[4] = cate_reviews[-2].text.strip()
    else:
        cate_reviews.insert(-2 , "Missing")


    if review[i].find_all('td' , class_="review-rating-header recommended"):
      features[5] = cate_reviews[-1].text.strip()
    else:
        cate_reviews.insert(-1 , "Missing")

    details['Aircraft'].append(features[0])
    details['Type Of Traveller'].append(features[1])
    details['Seat Type'].append(features[2])
    details['Route'].append(features[3])
    details['Date Flown'].append(features[4])
    details['Recommended'].append(features[5])


    cate_reviews.clear()

    # Get the numerical reviews
    num_reviews = review[i].find_all('span' , class_= 'star fill')
    for item in range(len(num_reviews)):
       num.append(num_reviews[item].text.strip())

    num=list(map(int,num))

    num = split_list_on_one(num)

    for rate in range(len(num)):
      rating.append(num[rate][-1])

    if not review[i].find_all('td' , class_="review-rating-header seat_comfort"):
      rating.insert(0 , 0)

    if not review[i].find_all('td' , class_="review-rating-header cabin_staff_service"):
      rating.insert(1 , 0)

    if not review[i].find_all('td' , class_="review-rating-header food_and_beverages"):
      rating.insert(2 , 0)

    if not review[i].find_all('td' , class_="review-rating-header inflight_entertainment"):
      rating.insert(3, 0)

    if not review[i].find_all('td' , class_="review-rating-header ground_service"):
      rating.insert(4 , 0)

    if not review[i].find_all('td' , class_="review-rating-header wifi_and_connectivity"):
      rating.insert(5 , 0)

    if not review[i].find_all('td' , class_="review-rating-header value_for_money"):
      rating.insert(6 , 0)

    details['Seat Comfort'].append(rating[0])
    details['Cabin Staff Service'].append(rating[1])
    details['Food & Beverages'].append(rating[2])
    details['Inflight Entertainment'].append(rating[3])
    details['Ground Service'].append(rating[4])
    details['Wifi & Connectivity'].append(rating[5])
    details['Value For Money'].append(rating[-1])


    rating.clear()
    num.clear()

# Saving the output in csv file

In [85]:
df = pd.DataFrame(details)

In [86]:
df.to_csv("reviews.csv", index=True)